In [ ]:
# ================================================
# 🧩 STEP 1: Install Dependencies
# ================================================
!pip install -q transformers peft accelerate datasets bitsandbytes

In [2]:
# ================================================
# 🧩 STEP 2: Import Libraries
# ================================================
import torch
#from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
# ================================================
# 🧩 STEP 3: Load data set
# ================================================

from datasets import load_dataset
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Colab Notebooks/boot_recommendations.json")

print(dataset)

dataset = dataset["train"].train_test_split(test_size=0.2)

print(dataset)



In [ ]:

# ================================================
# 🧩 STEP 4: Load Base Model and Tokenizer
# ================================================
model_name = "facebook/opt-1.3b"  # You can use "facebook/opt-350m" for Colab CPU
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
# ================================================
# 🧩 STEP 5: Tokenize Data
# ================================================
def format_example(example):
    return f"### Instruction:\n{example['instruction']}\n### Response:\n{example['response']}"

def tokenize_function(example):
    text = format_example(example)
    tokenized = tokenizer(text, truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_datasets = dataset.map(tokenize_function, batched=False)

In [ ]:
# ================================================
# 🧩 STEP 6: Prepare LoRA Configuration
# ================================================
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],  # For OPT models
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

In [ ]:
# ================================================
# 🧩 STEP 7: Load Model and Apply LoRA
# ================================================
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,  # Optional, for memory efficiency
    device_map="auto"
)
model = get_peft_model(model, lora_config)

In [ ]:
# ================================================
# 🧩 STEP 8: Training Arguments
# ================================================
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    learning_rate=2e-4,
    num_train_epochs=3,
    logging_steps=10,
    save_total_limit=1,
    eval_strategy="steps",
    fp16=True,
)

In [8]:
# ================================================
# 🧩 STEP 9: Trainer Setup
# ================================================
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

In [ ]:
trainer.train()

In [ ]:
# ================================================
# 🧩 STEP 10: Save Fine-Tuned Model
# ================================================
trainer.save_model("./lora-boot-recommendation-finetuned-model")
tokenizer.save_pretrained("./lora-boot-recommendation-finetuned-model")


In [ ]:
# ================================================
# 🧩 STEP 11: Test the Model
# ================================================
from transformers import pipeline

pipe = pipeline("text-generation", model="./lora-boot-finetuned-model", tokenizer=tokenizer, max_new_tokens=100)

prompt = "### Instruction:\nWhat boot do you recommend for winter?\n### Response:\n"
result = pipe(prompt)
print(result[0]['generated_text'])

In [ ]:
# ================================================
# 🧩 STEP 11: Test the Model
# ================================================
from transformers import pipeline

pipe = pipeline("text-generation", model="./lora-finetuned-model", tokenizer=tokenizer, max_new_tokens=100)

prompt = "### Instruction:\nHow do I update my shipping address?\n### Response:\n"
result = pipe(prompt)
print(result[0]['generated_text'])

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import userdata
hf_token = userdata.get("HF_TOKEN")
from huggingface_hub import login
login(hf_token)

model_name = "./boot-recommendation-finetuned-model"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

model.push_to_hub("sichilam/boot-recommendation-finetuned-model")
tokenizer.push_to_hub("sichilam/boot-recommendation-finetuned-model")